# MarsToday: evaluating Mars climate through REMS sensor onboard Curiosity Mars rover

## Import box

In [169]:
#Imports
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
from pandas import json_normalize
import re
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
load_dotenv()

True

## Data first visualization

In [3]:
weather = pd.read_csv("../data/mars-weather.csv")
weather

,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,wind_speed,atmo_opacity
0,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,NaN,Sunny
1,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,NaN,Sunny
2,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,NaN,Sunny
3,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,NaN,Sunny
4,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,NaN,Sunny
...,...,...,...,...,...,...,...,...,...,...
1889,24,2012-08-18,12,156,Month 6,-76.0,-18.0,741.0,NaN,Sunny
1890,13,2012-08-17,11,156,Month 6,-76.0,-11.0,740.0,NaN,Sunny
1891,2,2012-08-16,10,155,Month 6,-75.0,-16.0,739.0,NaN,Sunny
1892,232,2012-08-15,9,155,Month 6,NaN,NaN,NaN,NaN,Sunny


## Connection with NASA API

In [158]:
# This is a function to call NASA api with a given value of Sol day and Camera type.

def call_Curiosity (date, camera):
    """
    This is a function that calls NASA API 'Mars Rover Photos' with two arguments. It returns the url from
    a specific camera onboard Curiosity rover.
    date: input the desired date in the format YYYY-MM-DD as a STRING,
    camera: select between FHAZ, RHAZ, MAST, CHEMCAM, MAHLI, MARDI, NAVCAM, PANCAM, MINITES, as STRING
    
    """
        
    try:
        nasa = os.getenv("token")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={date}&camera={camera}&api_key={nasa}"
        request = requests.get(url)
        df = pd.DataFrame(request.json())
        df_clean = pd.DataFrame(df.values[0][0])
        image_url = list(df_clean["img_src"])[0]

        return f"Image available for camera {camera} onboard Curiosity rover: {image_url}"

    except:
            
        return f"No image available on {date} for camera {camera} onboard Curiosity rover, please select another Sol day"    

## Getting Mars pictures

In [161]:
def get_pictures_Curiosity(date):
    
    """
    This is a function that calls call_NASA function with one argument. It returns the url of all the pictures 
    taken by all the cameras of Curiosity rover from a specific Sol date.
    date: input the desired date in the format YYYY-MM-DD as STRING.
    
    """       
    cameralist = ["FHAZ", "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]
    for i in cameralist:
        print(call_Curiosity(date, i))
    pass

# Data cleaning

In [155]:
def remove_columns(df, column_name):
    
    df.drop(columns=f"{column_name}", inplace=True)
    
    return df

In [15]:
remove_columns(weather, "wind_speed")

,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,atmo_opacity
1893,1,2012-08-07,1,150,Month 6,NaN,NaN,NaN,Sunny
1892,232,2012-08-15,9,155,Month 6,NaN,NaN,NaN,Sunny
1891,2,2012-08-16,10,155,Month 6,-75.0,-16.0,739.0,Sunny
1890,13,2012-08-17,11,156,Month 6,-76.0,-11.0,740.0,Sunny
1889,24,2012-08-18,12,156,Month 6,-76.0,-18.0,741.0,Sunny
...,...,...,...,...,...,...,...,...,...
4,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,Sunny
3,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,Sunny
2,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,Sunny
1,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,Sunny


In [186]:
weather["mean_temp"] = ((weather["min_temp"] + weather["max_temp"])/2)

In [188]:
max_temp = px.line(weather[["sol", "min_temp", "max_temp", "mean_temp"]], x="sol", y=["max_temp", "min_temp", "mean_temp"])
max_temp.show()